# JSON and CSV: Working with Structured Data in Python

In this presentation, we'll learn how to:
1. Save and load nested data using `json`
2. Read and write tabular data using `csv`
3. Know when to use each

### Motivation: Sharing data between programs
Suppose we wrote a Python script that collects user survey results.
We want to:
- Send them to a web app
- Share with your boss using Excel

What format can we use?

Humans and machines often exchange data in JSON or CSV because both are:
- plain-test formats (easy to read, share, and version control)
- widely supported across languages

## What JSON and CSV Formats look like

### JSON Example
JSON (*JavaScript Object Notation*) represents structured, hierarchical data:

```python
{
  "name": "John",
  "age": 30,
  "isStudent": false,
  "courses": ["Math", "Science", "History"]
}
```
### CSV Example
CSV (*Comma Separated Values*) represents flat and tabular data:

```python
Name,Age,City
John Doe,30,New York
Jane Smith,25,Los Angeles
```



## JSON module in Python: 

The `json` module allows Python to communicate with systems that use JSON.

**Two main operations:**
- *Encoding*: Python → JSON  
  - `json.dumps(obj)` → string  
  - `json.dump(obj, file)` → write to file
- *Decoding*: JSON → Python  
  - `json.loads(str)` → parse from string  
  - `json.load(file)` → read from file


In [5]:
import json

#1. Your survey data ("What's your name, your age, and the pet you have?")
data = {
    "users": [
        {"name": "Jess", "age": 22, "pets": ["dog", "cat", "fish", "shrimp"]},
        {"name": "Chelsea", "age": 25, "pets": ["cat"]}
    ]
}

#2. Write the data to a JSON file
with open("users.json", "w") as f:
    json.dump(data, f, indent=2)

#3. Read the JSON back
with open("users.json", "r") as f:
    users = json.load(f)

print("First user:", users["users"][0]["name"])

#additional functionalities:
#We can make it format it nicely

print(json.dumps(data, indent = 2, sort_keys = True))

First user: Jess
{
  "users": [
    {
      "age": 22,
      "name": "Jess",
      "pets": [
        "dog",
        "cat",
        "fish",
        "shrimp"
      ]
    },
    {
      "age": 25,
      "name": "Chelsea",
      "pets": [
        "cat"
      ]
    }
  ]
}


### Real-World Example: Using JSON from an API (Wikipedia)

Most web APIs like wikipedia send and receive data in JSON format.  
Here’s how we can request information about a Wikipedia page and parse it in Python.

In [13]:
import requests
import json

headers = {
    "User-Agent": "json-csv-demo/1.0 (contact: jxiong3@uchicago.edu)"
}

#Wikipedia REST API
url = "https://en.wikipedia.org/api/rest_v1/page/summary/Python_(programming_language)"
response = requests.get(url, headers=headers)

#Raw JSON text
print("Raw JSON response:")
print(response.text[:300] + "...")

# Parse into Python dict
data = response.json()

print("\nParsed keys:")
print(list(data.keys()))

print("\nTitle:", data["title"])
print("Description:", data["description"])
print("Extract (first sentence):", data["extract"][:120], "...")

Raw JSON response:
{"type":"standard","title":"Python (programming language)","displaytitle":"<span class=\"mw-page-title-main\">Python (programming language)</span>","namespace":{"id":0,"text":""},"wikibase_item":"Q28865","titles":{"canonical":"Python_(programming_language)","normalized":"Python (programming language...

Parsed keys:
['type', 'title', 'displaytitle', 'namespace', 'wikibase_item', 'titles', 'pageid', 'thumbnail', 'originalimage', 'lang', 'dir', 'revision', 'tid', 'timestamp', 'description', 'description_source', 'content_urls', 'extract', 'extract_html']

Title: Python (programming language)
Description: General-purpose programming language
Extract (first sentence): Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the ...


### Additional JSON features

- **Pretty printing** (`indent`, `sort_keys`) — make files human-readable.  

- **Unicode handling** (`ensure_ascii=False`) — keeps non-English characters visible.  

- **Error handling** (`JSONDecodeError`) — helps pinpoint bad JSON input - useful for validating data from others 

- There are many other cool features


## CSV module in Python:

The `csv` module supports reading and writing comma-separated files.

**Main interfaces:**
- `csv.writer` → writes lists
- `csv.reader` → reads rows as lists
- `csv.DictWriter` / `csv.DictReader` → use column names


In [14]:
import csv
rows = [
    ["name", "item", "price"],
    ["Jess", "apple", 1.2],
    ["Rob", "banana", 0.8]
]

#Write the CSV
with open("sales.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

#Read the CSV
with open("sales.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)


['name', 'item', 'price']
['Jess', 'apple', '1.2']
['Rob', 'banana', '0.8']


### Other CSV Features

### DictReader and DictWriter

Normally, `csv.reader` and `csv.writer` work with **lists** — each row is just a list of values.  
But sometimes it’s easier to work with column names instead of remembering column positions.

That’s what `DictReader` and `DictWriter` do:

- **`csv.DictWriter`** — writes each row from a dictionary (keys = column names).  

In [18]:
#Write using DictWriter
with open("sales_dict.csv", "w", newline="") as f:
    fieldnames = ["name", "item", "price"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({"name": "Jess", "item": "apple", "price": 1.2})
    writer.writerow({"name": "Rob", "item": "banana", "price": 0.8})

#Read using DictReader
totals = {}
with open("sales_dict.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row["name"], "bought", row["item"], "for", row["price"])
        totals[row["name"]] = totals.get(row["name"], 0) + float(row["price"]) #compute total sales per person
print("Total sales by person:")
for name, total in totals.items():
    print(name, "→ $", round(total, 2))

Jess bought apple for 1.2
Rob bought banana for 0.8
Total sales by person:
Jess → $ 1.2
Rob → $ 0.8



- **Custom delimiters** (`delimiter="\t"`, `";"`, etc.) — handles files with different separators (that are not commas)

- **Quoting and escape rules** (`quotechar`, `escapechar`) — manage commas or quotes inside text fields.  

### Summary
- Use **JSON** for nested or hierarchical data (APIs, configs).  
- Use **CSV** for flat, tabular data (spreadsheets, analytics).  
